In [16]:
using Revise
using FFTW, Optim, Images, ImageView, TestImages
using Noise
using DeconvOptim
using Statistics
using BenchmarkTools
using Deconvolution
using Zygote
#using ForwardDiff
#using ReverseDiff
#using DiffRules
using Plots
#BenchmarkTools.DEFAULT_PARAMETERS.samples = 10
#BenchmarkTools.DEFAULT_PARAMETERS.seconds = 100

In [17]:
function imshowv(args...)
    imshow(cat(args..., dims=3))
end


function lucy2(observed::AbstractArray, psf::AbstractArray; iterations::Integer = 1)
    @assert size(observed) == size(psf)
    @assert iterations >= 0

    psf_ft = fft(psf)
    psf_ft_conj = conj.(psf_ft)

    function lucystep(e)
        return e .* real(ifft(fft(observed ./ ifft(fft(e) .* psf_ft)) .* psf_ft_conj)) 
    end

    estimated = real(ifft(fft(observed) .* psf_ft))
    for i in 1:iterations
        estimated = lucystep(estimated)
    end

    return estimated
end

lucy2 (generic function with 1 method)

In [18]:
img = convert(Array{Float32}, channelview(load("../DeconvOptim/resolution_512.tif")))
#img = convert(Array{Float32}, testimage("fabio_gray_512"))
#img = convert(Array{Float32}, channelview(testimage("peppers_gray"))[1, :, :])
r = 30
psf = convert(Array{Float32}, DeconvOptim.generate_psf(img, r))
otf = rfft(psf)

img_b = max.(0, DeconvOptim.conv_real_otf(img, otf))
img_noisy = poisson(img_b, 100)

imshowv(img, img_b, img_noisy);

In [19]:
@time res_lr1 = lucy(img_noisy, psf, iterations=100);
@time res_lr2 = lucy2(img_noisy, psf, iterations=100);

  3.505301 seconds (1.74 M allocations: 1.844 GiB, 2.91% gc time)
  2.649418 seconds (215.08 k allocations: 1.781 GiB, 2.65% gc time)


In [120]:
@time resTikho_sgs, o, r0 = deconvolution(img_noisy, psf, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=Tikhonov(λ=0.03, mode="spatial_grad_square"),
        options=Optim.Options(iterations=10))
print(o)


@time resTikho_for, o, r0 = deconvolution(img_noisy, psf, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=Tikhonov(λ=0.02, mode="forward_grad"),
        options=Optim.Options(iterations=10))
print(o)


@time resTikho_cen, o, r0 = deconvolution(img_noisy, psf, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=Tikhonov(λ=0.02, mode="central_grad"),
        options=Optim.Options(iterations=10))
print(o)

  1.057051 seconds (989.59 k allocations: 831.294 MiB, 5.06% gc time)
 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [-3.07e-04, -2.56e-06, 5.54e-04,  ...]
    Minimum:   2.054031e-01

 * Found with
    Algorithm:     L-BFGS
    Initial Point: [4.68e-02, 4.71e-02, 4.77e-02,  ...]

 * Convergence measures
    |x - x'|               = 3.00e-02 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.81e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 3.19e-06 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.55e-05 ≰ 0.0e+00
    |g(x)|                 = 9.64e-08 ≰ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    10
    f(x) calls:    37
    ∇f(x) calls:   37
  0.612795 seconds (14.01 k allocations: 931.595 MiB, 4.56% gc time)
 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [-6.78e-04, -2.72e-04, 5.16e-04,  ...]
    Minimum:   2.052740e-01

 * Found with
    Algorithm:     L-BFGS
    Initial

In [121]:
imshowv(img, resTikho_sgs[:, :], resTikho_for[:, :], resTikho_cen[:, :], res_lr2, img_noisy)

Dict{String,Any} with 4 entries:
  "gui"         => Dict{String,Any}("window"=>GtkWindowLeaf(name="", parent, wi…
  "roi"         => Dict{String,Any}("redraw"=>330: "map(clim-mapped image, inpu…
  "annotations" => 279: "input-117" = Dict{UInt64,Any}() Dict{UInt64,Any} 
  "clim"        => 278: "CLim" = CLim{Float32}(0.0, 1.0) CLim{Float32} 

In [93]:
@time resTV_old, o, r0 = deconvolution(img_noisy, psf, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=TV_old(λ=0.02),
        options=Optim.Options(iterations=10))
print(o)


@time resTV, o, r0 = deconvolution(img_noisy, psf, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=TV(λ=0.02, mode="forward"),
        options=Optim.Options(iterations=10))
print(o)


  1.236869 seconds (764.22 k allocations: 746.583 MiB, 2.78% gc time)
 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [4.70e-03, 6.20e-03, 7.54e-03,  ...]
    Minimum:   2.056203e-01

 * Found with
    Algorithm:     L-BFGS
    Initial Point: [5.17e-02, 5.20e-02, 5.23e-02,  ...]

 * Convergence measures
    |x - x'|               = 2.34e-02 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.23e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 5.14e-06 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.50e-05 ≰ 0.0e+00
    |g(x)|                 = 3.72e-07 ≰ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    10
    f(x) calls:    37
    ∇f(x) calls:   37


InterruptException: InterruptException:

In [94]:
imshowv(img, resTV_old[:,:], resTV[:, :], res_lr2,  img_noisy)

UndefVarError: UndefVarError: resTV not defined

In [ ]:
img3d = cat(img_noisy, img_noisy, img_noisy, dims=3);
img4d = cat(img3d, img3d ./ 2, img3d .* 2, dims=4);
img5d = reshape(img4d, (size(img4d)..., 1))

psf3d = cat(psf, zeros(size(img)), zeros(size(img)), dims=3)
psf4d = cat(psf, psf, psf, dims=3)
psf5d = reshape(psf4d, (size(psf4d)..., 1))
#imshow(img3d)
print(size(psf5d))
print(size(img5d))


@time resGR, o, r0 = deconvolution(img5d, psf3d, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=DeconvOptim.GR(λ=0.02, mode="central"),
        options=Optim.Options(iterations=10))
print(o)

@time resGR_old, o, r0 = deconvolution(img5d, psf3d, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=DeconvOptim.GR(λ=0.02, mode="forward"),
        options=Optim.Options(iterations=10))
print(o)


(512, 512, 3, 1)(512, 512, 3, 3, 1) 11.880658 seconds (5.05 M allocations: 8.661 GiB, 12.89% gc time)
 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [1.29e-03, 2.66e-03, 3.95e-03,  ...]
    Minimum:   2.110065e-01

 * Found with
    Algorithm:     L-BFGS
    Initial Point: [5.17e-02, 5.20e-02, 5.23e-02,  ...]

 * Convergence measures
    |x - x'|               = 1.50e-01 ≰ 0.0e+00
    |x - x'|/|x'|          = 9.21e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.64e-06 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 7.77e-06 ≰ 0.0e+00
    |g(x)|                 = 2.61e-06 ≰ 1.0e-08

 * Work counters
    Seconds run:   8  (vs limit Inf)
    Iterations:    10
    f(x) calls:    37
    ∇f(x) calls:   37


In [ ]:
imshowv(img, resGR_old[:,:,1,1,1], resGR[:,:,2,1,1], res_lr2,  img_noisy)

In [ ]:
using TestImages
using Tullio
img = convert(Array{Float32}, testimage("fabio_gray_512"))
x = reshape(img, 512, 512, 1)

@tullio res_t = (x[i + 1, j, k] + x[i - 1, j, k+1])^2 
print(res_t, "\n")

@tullio res_t = (x[i + 1, j, k] + x[i - 1, j, k - 1])^2
print(res_t, "\n")

res_f = 0
for j = 1:size(x)[2]
    for i = 2:size(x)[1] - 1
        res_f += (x[i+1, j] + x[i - 1, j])^2
    end
end
print(res_f)

In [ ]:
219891.22
219891.22
219914.31

In [ ]:
x = randn(500, 500);

function ∇s_square5(img)
    ∇y = img[3:end, 2:end - 1] .- img[1:end - 2, 2:end - 1]
    ∇x = img[2:end - 1, 3:end] .- img[2:end - 1, 1:end - 2]
    return (∇x .^2 .+ ∇y .^2) ./ 4 
end

function ∇spatial_square2(rec; ϵ=1e-5)
    out = similar(rec)#zeros(size(rec))
    R = CartesianIndices(rec)
    c_first, c_last = first(R), last(R)
    uc = oneunit(c_first)
    for I in R
        n, s = 0, 0#zero(eltype(out))
        for J in max(c_first, I - uc):min(c_last, I + uc)
            n += 1
            s += (rec[I] - rec[J])^2
        end
        out[I] = s / n
    end
    return out ./ 4 ./ length(rec)
end


#@time ∇spatial_square2(x)
#@time ∇s_square5(x)
#@time ForwardDiff.gradient(rec -> sum(∇spatial_square2(rec)), x)
#@time ReverseDiff.gradient(rec -> sum(∇s_square5(rec)), x)
#@time ForwardDiff.gradient(rec -> sum(∇s_square5(rec)), x)
#@time ForwardDiff.gradient(rec -> sum(∇s_square5(rec)), x)
#@time ReverseDiff.gradient(rec -> sum(∇s_square5(rec)), x)
#@time ReverseDiff.gradient(rec -> sum(∇spatial_square2(rec)), x)
#@time Zygote.gradient(rec -> sum(∇spatial_square2(rec)), x)
#@time ReverseDiff.gradient(rec -> sum(∇s_square5(rec)), x)
@time Zygote.gradient(rec -> sum(∇s_square5(rec)), x)
@time Zygote.gradient(rec -> sum(∇s_square5(rec)), x)
@time sum(∇s_square5(x))
@time sum(∇s_square5(x))
#@time Zygote.gradient(rec -> sum(abs.(fft(rec))), x)
#@time ReverseDiff.gradient(rec -> sum(abs.(fft(rec))), x)
#@time ∇_∇s_square(x)

In [ ]:
function boxcar3(A::AbstractArray)
    out = similar(A)
    R = CartesianIndices(A)
    Ifirst, Ilast = first(R), last(R)
    I1 = oneunit(Ifirst)
    for I in R
        n, s = 0, zero(eltype(out))
        for J in max(Ifirst, I-I1):min(Ilast, I+I1)
            s += A[J]
            n += 1
        end
        out[I] = s/n
    end
    out
end

boxcar3(x)

In [ ]:
using ImageFiltering
using Statistics
img_m = mapwindow(median!, img_noisy, (2,2))

In [ ]:
imshowv(img, img_m, img_noisy)

In [ ]:
f, df, inv_f = Non_negative2()
x = -5:0.01:5
#plot(x, f.(inv_f.(x)))
#plot(x, inv_f.(f.(x)))
#plot(x, inv_f.(x))
#plot!(x, df(x))
plot(x, f.(x))
plot!(x, inv_f.(x), ylims=(-5,5 ))

In [ ]:
#mg1img2 = convert(Array{Float32}, channelview(testimage("peppers_gray"))[1, :, :])
img1 = img
img2 = img_b 
img3 = img_noisy 
img4 = resGR2
img1_ft = abs.(fftshift(fft(img1)))
img2_ft = abs.(fftshift(fft(img2)))
img3_ft = abs.(fftshift(fft(img3)))
img4_ft = abs.(fftshift(fft(img4)))
N, _ = size(img1)
plot(img1_ft[N ÷ 2:400, N ÷ 2 + 1])
plot!(img2_ft[N ÷ 2:400, N ÷ 2 + 1])
plot!(img4_ft[N ÷ 2:400, N ÷ 2 + 1])
#imshowv(img1_ft, img2_ft, img3_ft, img4_ft)

In [ ]:
imshowv(img3, img2)

In [ ]:
using Tullio, Einsum, LoopVectorization

function laplace_vec(rec)                                                           
    @views a = (rec[1:end-2, 2:end-1] .- 4 .* rec[2:end - 1, 2:end - 1]                
                .+ rec[3:end, 2:end - 1])                                           
    @views b = (rec[2:end-1, 1:end-2]                 
                .+ rec[2:end - 1, 3:end])                                           
                                                                                
    return @views sum((a .+ b) .^ 2)                                                   
end     


function laplace_for(rec)
    res = zero(eltype(rec))
    @avx for j = 2:size(rec)[2] - 1
        for i = 2:size(rec)[1] - 1
            @inbounds res += (rec[i - 1, j] + rec[i+1, j] 
                              + rec[i, j+1] + rec[i, j-1] - 4 * rec[i,j])^2
        end
    end
    return res
end
        

laplace_tul(rec) = @tullio res = (rec[i - 1, j] + rec[i + 1, j] +   
                                  rec[i, j + 1] + rec[i, j - 1] - 4 * rec[i,j])^2


In [ ]:
x = convert(Array{Float32}, 10 .+ randn(500, 500))
@time a = laplace_vec(x)
@time a = laplace_vec(x)
@time b = laplace_for(x)
@time b = laplace_for(x)
@time c = laplace_tul(x)
@time c = laplace_tul(x)


vec_tape_comp = ReverseDiff.compile(ReverseDiff.GradientTape(laplace_vec, x))
for_tape_comp = ReverseDiff.compile(ReverseDiff.GradientTape(laplace_for, x))

print("Zygote", "\n")
@time Zygote.gradient(laplace_vec, x)
@time Zygote.gradient(laplace_vec, x)
@time Zygote.gradient(laplace_tul, x)
@time Zygote.gradient(laplace_tul, x)
#@time ReverseDiff.gradient!(similar(x), vec_tape_comp, x);
#@time ReverseDiff.gradient!(similar(x), vec_tape_comp, x);
#@time ReverseDiff.gradient!(similar(x), for_tape_comp, x);
#@time ReverseDiff.gradient!(similar(x), for_tape_comp, x);
##@time Zygote.gradient(laplace_for, x)
#@time Zygote.gradient(laplace_for, x)

a ≈ b
a ≈ c

In [ ]:
print(a, "\n", b, "\n", c)

In [ ]:
function L2norm_v(x)
    return sum(x .^ 2)
end

function L2norm_f(x)
    res = 0
    for j = 1:size(x)[2]
        for i = 1:size(x)[1]
            @inbounds res += x[i, j]^2
        end
    end
    return res
end

function L2norm_df(x)
    res = 0
    for j = 1:size(x)[2]
        for i = 1:size(x)[1]
            @inbounds res += 2 .* x[i, j]
        end
    end
    return res
end   

function L2norm_dv(x)
    return 2 .* x
end

x = rand(110, 100)

@time a = L2norm_v(x)
@time a = L2norm_v(x)
@time b = L2norm_f(x)
@time b = L2norm_f(x)
 
print("Zygote", "\n")
@time Zygote.gradient(L2norm_v, x)
@time Zygote.gradient(L2norm_v, x)
@time Zygote.gradient(L2norm_f, x)
@time Zygote.gradient(L2norm_f, x)

print("Analytic", "\n")
@time L2norm_dv(x)
@time L2norm_dv(x)
@time L2norm_df(x)
@time L2norm_df(x)


In [ ]:
M, K, N = 47, 73, 7;
A = rand(M, K);
b = rand(K);
c = rand(M);
d = rand(1,K,N);

E1 = Array{Float64}(undef, M, K, N);
E2 = similar(E1);

@time @. $E1 = exp($A - $b' +    $d) * $c
ldad = LowDimArray{(false,true,true)}(d);

@time @avx @. $E2 = exp($A - $b' + $ldad) * $c;

In [ ]:
E2 ≈ E1

In [ ]:
img3d = cat(img, dims=3);
img4d = cat(img3d, img3d ./ 2, img3d .* 2, dims=4);
img5d = cat(img4d, img4d ./ 2, img4d .* 2, dims=5);
#imshow(img3d)
print(size(img5d))

In [ ]:
y=rfft(img5d, [1, 2, 3]);
print(size(y))

In [ ]:
imshow(img5d[:, :, :, :, 1])

In [ ]:
imshow(real.(y[:, :, 1, :, 1]))

In [ ]:
img3d_2 = randn(size(img3d))
P = plan_rfft(img3d);
img3d_fft = P * img3d
P_inv = plan_irfft(img3d_fft, size(img3d)[1]);

img3d ≈ P_inv * (P * img3d)

img3d_2 ≈ P_inv * (P * img3d_2)

#size(P * img3d)
#size(P_inv * (P * img3d))

In [ ]:
using Tullio
using Zygote
arr_1 = randn((10, 10));

function f(arr)
    @tullio res1 = arr[i, k] - arr[i-1, k]
    @tullio res2 = arr[i, k] - arr[i, k+1]
    return res1 + res2
end

f(arr_1)

print(gradient(f, arr_1))

In [ ]:
using Tullio
arr = randn((10, 10))
#@tullio res = getindex(arr, [i, j])
@tullio res = getindex(arr, i, j) 
#@tullio res = arr[i, j] 
 
print(getindex(arr, 1, 2), " ", arr[1, 2])

#print(sum(arr), "\n", sum(arr))

In [62]:
using Tullio 

expr(d, inds1, inds2) = :(weights[$d] * abs2(arr[$(inds1...)] - arr[$(inds2...)]))
@eval function f(arr, weights)
    $(create_Ndim_regularizer(expr, identity, Ndim=3)...)
end

f (generic function with 1 method)

In [64]:
arr = randn((10, 10, 20))
@time f(arr, [1, 1, 1])

  0.000030 seconds (35 allocations: 1.375 KiB)


9845.955023367394

In [37]:
10 + -1

9

In [45]:
create_Ndim_regularizer(expr, identity, Ndim=2)


3-element Array{Any,1}:
 :(res_1 = let
          #= /home/fxw/Documents/Uni/FSU/2semester/Internship/DeconvOptim.jl/src/regularizer.jl:36 =#
          weights[1] * abs2(arr[i1 + 1, i2] - arr[i1 + -1, i2])
      end)
 :(res_2 = let
          #= /home/fxw/Documents/Uni/FSU/2semester/Internship/DeconvOptim.jl/src/regularizer.jl:36 =#
          weights[2] * abs2(arr[i1, i2 + 1] - arr[i1, i2 + -1])
      end)
 :(comb_f(res_1 + res_2))

In [87]:
x = ones((100, 200, 300, 20))
@time lol(x)

  0.249144 seconds (463 allocations: 31.000 KiB)


0.0

In [88]:
100 * 200 * 300 * 20 / 512 / 512

457.763671875

In [35]:
using Tullio
arr = randn((10, 10, 1))
@tullio res = arr[i,j, k]

15.494488004239969

In [29]:
function lol(num_dim, sum_dim)
    out, add = [], []
    for d in 1:sum_dim
        ind = :i # @gensym ind
        inds1 = map(1:num_dim) do di
            i = Symbol(ind, di)
            di == d ? :($i+1) : i
        end
        inds2 = map(1:num_dim) do di
            i = Symbol(ind, di)
            di == d ? :($i-1) : i
        end
        push!(add, :(arr[$(inds1...)] - arr[$(inds2...)]))
    end
    push!(out, :(@tullio res = +($(add...))))
    quote
        $(out...)
    end
end

lol (generic function with 2 methods)

In [61]:
@time @eval function lol2(arr)
    $(lol(3, 2))
end

  0.061565 seconds (9.53 k allocations: 549.306 KiB)


lol2 (generic function with 1 method)

In [34]:
@time lol2(arr)

  0.000015 seconds (12 allocations: 432 bytes)


-8.437124952164748

In [63]:
@time @eval function f(x)
    $(lol(3, 2))
end

  0.075221 seconds (9.53 k allocations: 549.134 KiB)


f (generic function with 1 method)

In [64]:
for i in 1:10
    for j in i:10
        

quote
    #= In[29]:17 =#
    #= In[29]:15 =# @tullio res = (arr[i1 + 1, i2, i3] - arr[i1 - 1, i2, i3]) + (arr[i1, i2 + 1, i3] - arr[i1, i2 - 1, i3])
end

In [36]:
using Revise
using DeconvOptim
using Random
@time reg = generate_spatial_grad_square_n(5, 3)

  0.011038 seconds (6.93 k allocations: 391.549 KiB)


#835 (generic function with 1 method)

In [41]:
Random.seed!(42)
arr = randn((3,3,3,3,3))
Zygote.gradient(reg, arr)
@time reg(arr)

ErrorException: no gradient definition here!

In [71]:
                                                                                
function create_Ndim_regularizer2(expr, num_dim, sum_dim)                       
    out, add = [], []                                                           
    for d in 1:sum_dim                                                          
        ind = :i # @gensym ind                                                  
        inds1 = map(1:num_dim) do di                                            
            i = Symbol(ind, di)                                                 
            di == d ? :($i+1) : i                                               
        end                                                                     
        inds2 = map(1:num_dim) do di                                            
            i = Symbol(ind, di)                                                 
            di == d ? :($i-1) : i                                               
        end                                                                     
        push!(add, expr(inds1, inds2))                                    
    end                                                                         
    push!(out, :(@tullio res = +($(add...))))                                   
    return out                                                                  
    #= quote =#                                                                 
    #=     $(out...) =#                                                         
    #= end =#                                                                   
end  

┌ Warning: Due to a previously reported error, the running code does not match saved version for the following files:
│ 
│   /home/fxw/Documents/Uni/FSU/2semester/Internship/DeconvOptim.jl/src/regularizer.jl
│ 
│ Use Revise.errors() to report errors again.
└ @ Revise /home/fxw/.julia/packages/Revise/tV8FE/src/Revise.jl:804


create_Ndim_regularizer2 (generic function with 1 method)

In [39]:
gradient(identity, 10)

(1,)

In [109]:
using Zygote
using Tullio

x = ones((5,5))

function f_working(arr)
    @tullio res = sqrt(arr[i, j])
    return res
end

foo = :sqrt 


function f_not_working(arr)
    @tullio res = $foo(arr[i, j])
    return res
end

f_working(x)
gradient(f_not_working, x)

MethodError: MethodError: objects of type Symbol are not callable